<a href="https://colab.research.google.com/github/bmd94/Churn-detection/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CHURN**

In [ ]:
import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
#%matplotlib qt
import seaborn as sns

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Kaggle/cell2celltrain.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Kaggle/cell2cellholdout.csv')

In [ ]:
'''import glob
import os
import pandas as pd   
df = pd.concat(map(pd.read_csv, glob.glob(os.path.join('/content/drive/MyDrive/Kaggle/cell', "*.csv"))))
df.head()'''

In [ ]:
df_train.head()

In [ ]:
df_train.columns

In [ ]:
print(df_train.shape,'  ',df_test.shape)

In [ ]:
df_train.dtypes

In [ ]:
(df_train.dtypes == 'object').sum()

In [ ]:
df_train.isna().any().sum()

In [ ]:
df_train.info()

In [ ]:
df_train.describe().T

In [ ]:
df_rt = df_train.fillna(0)

In [ ]:
df_rt.isna().any().sum()

In [ ]:
df_test.isna().any().sum()

In [ ]:
df_rt.nunique()

In [ ]:
df_rt.dtypes=='object' 

In [ ]:
col = []
for i in df_rt.columns:
    if df_rt[i].dtype=='object':
        col.append(i)
col

In [ ]:
cat= col[1:]
len(cat)                 
cat

In [ ]:
print('size of dataframe befor onehot encoder',df_rt.shape)
for c in cat:
    # use pd.concat to join the new columns with your original dataframe
    df_rt = pd.concat([df_rt,pd.get_dummies(df_rt[c], prefix=c)],axis=1)

    # now drop the original 'country' column (you don't need it anymore)
    df_rt.drop([c],axis=1, inplace=True)
print('size of dataframe after onehot encoder',df_rt.shape)

In [ ]:
df_rt.head()

In [ ]:
(df_rt.dtypes=='object').sum() 

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
df_rt['Churn']= le.fit_transform(df_rt['Churn'])

In [ ]:
df_rt.head()

In [ ]:
(df_rt.dtypes=='object').sum() 

In [ ]:
X = df_rt.iloc[:,2:]
y = df_rt['Churn']
print(X.shape,y.shape)

In [ ]:
(X.dtypes=='object').sum() 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)
y_test.shape,y_train.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
pip install xgboost

In [ ]:
pip install catboost

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [ ]:

classifiers = [['DecisionTree :',DecisionTreeClassifier()],
               ['RandomForest :',RandomForestClassifier()], 
               ['Naive Bayes :', GaussianNB()],
               ['KNeighbours :', KNeighborsClassifier()],
               #['SVM :', SVC()],
               ['Neural Network :', MLPClassifier()],
               ['LogisticRegression :', LogisticRegression()],
               ['ExtraTreesClassifier :', ExtraTreesClassifier()],
               ['AdaBoostClassifier :', AdaBoostClassifier()],
               ['GradientBoostingClassifier: ', GradientBoostingClassifier()],
               ['XGB :', XGBClassifier()],
               ['CatBoost :', CatBoostClassifier(logging_level='Silent')]]

predictions_df = pd.DataFrame()
predictions_df['actual_labels'] = y_test
accu=dict()
for name,classifier in classifiers:
    classifier = classifier
    classifier.fit(X_train, y_train)
    predictions = classifier.predict(X_test)
    predictions_df[name.strip(" :")] = predictions
    accu[name.replace(' :','')]=[round(accuracy_score(y_test, predictions),4)]
    print(name, accuracy_score(y_test, predictions))
    


In [ ]:
 accu

In [ ]:
from collections import OrderedDict
d_sorted_by_value = OrderedDict(sorted(accu.items(), key=lambda x: x[1],reverse=True))

In [ ]:
d_sorted_by_value

In [ ]:
mdl = pd.DataFrame.from_dict(d_sorted_by_value, orient='index', columns=['Accu'])
mdl

In [ ]:
plt.figure(figsize=(10, 8))
sns.set_theme(style="whitegrid")
ax = sns.barplot(x=mdl.index, y=mdl['Accu'], data=mdl)
plt.xticks(rotation=90)

In [ ]:
classifier = CatBoostClassifier(logging_level='Silent')
classifier.fit(X_train,y_train)
prediction = classifier.predict(X_test)
print('='*63)
print('Accuaracy of Model ',accuracy_score(y_test, prediction))
print('F1 score of Model ',f1_score(y_test, prediction))  # f1_score recall_score
print('Recall score of Model ',recall_score(y_test, prediction))  # f1_score precision_score
print('Precision score of Model ',precision_score(y_test, prediction))  # f1_score precision_score
print('='*63)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

classifier_cm= confusion_matrix(y_test,prediction)
f, ax = plt.subplots(figsize=(5,5))
sns.heatmap(classifier_cm, annot=True, linewidth=0.7, linecolor='black', fmt='g', ax=ax,
            cmap="BuPu")
plt.title('VotingClassification Confusion Matrix')
plt.xlabel('Predictions')
plt.ylabel('Y test')
plt.show()

In [ ]:
classifier_cm = classification_report(y_test,predictions)
print(classifier_cm)